In [45]:
!wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-04 07:03:58--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-04 07:03:59--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1b1aabfd1dca29f08b5f67b9ed.dl.dropboxusercontent.com/cd/0/inline/BN0dZFySrWhK7fc8Bqe8mZknm_xcAVVnYKks-nBiybX7amBN4ssMuf-hujVl1HGtkVjkSSWT5DJzlZ9eqDKiSjQ9IfdvOpODWCNivbk2DuF5uxGipIXwpo9GeuTwQjnWi-wUvP4ef5xmxGIURHwPa0S_/file# [following]
--2021-05-04 07:03:59--  https://uc1b1aabfd1dca29f08b5f67b9ed.dl.dropboxusercontent.com/cd/0/inline/BN0dZFySrWhK7fc8Bqe8mZknm_xcAVVnYKks-nBiybX7amBN4ssMuf-hujVl1

In [46]:
!nvidia-smi

Tue May  4 07:05:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    38W / 300W |   5987MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [47]:
import pickle

import torch
!pip install transformers
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
from scipy import stats
import sklearn
import collections
import random

from google.colab import drive

random.seed(2021)
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
# load sample data
# format { post_id: [post, [comment1, comment2, ... ] }

data_file_path = 'grouped_data.pickle'

def get_best_comm(comms):
  best_comm = None
  best_score = -np.inf

  for comm in comms:
    if comm['score'] > best_score:
      best_comm = comm
      best_score = comm['score']

    return best_comm

def load_data(file_path):
    posts_comments = []
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        for post_id, val in tqdm(list(data.items())[:5000]):
            post = val[0]
            comms = val[1]
            # best_comm = get_best_comm(comms)

            # if len(post['title']) < 500 and len(best_comm['body']) < 500 and best_comm['body'] != '[deleted]' and best_comm['body'] != '[removed]':
            #   posts_comments.append((post['title'], best_comm['body']))

            for com in comms:
                try:
                    if len(post['title']) < 500 and len(com['body']) < 500 and com['body'] != '[deleted]' and com['body'] != '[removed]':
                        posts_comments.append((post['title'], com['body']))
                except:
                    pass
                
    return posts_comments
    
posts_comms = load_data(data_file_path)
len(posts_comms)

65242

In [49]:
post_comms_realfake = []
labels = []
for post, comment in posts_comms:
  while True:
    rpost, rcomment = random.choice(posts_comms)
    if post != rpost:
      break
  
  post_comms_realfake.append((post, comment))
  labels.append(0) # comment \in post
  post_comms_realfake.append((post, rcomment))
  labels.append(1) # comment \nin post

len(post_comms_realfake)

130484

In [50]:
# split text
train_texts, test_texts, train_labels, test_labels = train_test_split(post_comms_realfake, labels, test_size=.25, shuffle=True)
# train_texts, test_texts, train_labels, test_labels = train_test_split(post_comms_realfake, labels, test_size=.01, train_size=.04, shuffle=True)

In [51]:
# load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', use_cache=True)
tokenizer.pad_token = tokenizer.eos_token

In [52]:
# tokenize train data

train_post = []
train_comm = []

for x in tqdm(train_texts):
    train_post.append(x[0])
    train_comm.append(x[1])

train_texts = None
train_encodings = tokenizer(text=train_post, text_pair=train_comm, truncation=True, padding=True)


In [53]:
# tokenize test data

test_post = []
test_comm = []

for x in tqdm(test_texts):
    test_post.append(x[0])
    test_comm.append(x[1])

test_texts = None

test_encodings = tokenizer(text=test_post, text_pair=test_comm, truncation=True, padding=True)


In [54]:
# load model
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.train()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
      

In [29]:
# create dataset class and load encodings and associated labels to it

class upvote_prediction_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = upvote_prediction_dataset(train_encodings, train_labels)
test_dataset = upvote_prediction_dataset(test_encodings, test_labels)

In [43]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=12,  # batch size per device during training
    per_device_eval_batch_size=12,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    logging_dir='./logs',            # directory for storing logs
    logging_steps = 10,
)

model.config.pad_token_id = tokenizer.pad_token_id

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    tokenizer = tokenizer
)


trainer.train()


Step,Training Loss
10,3.213900
20,2.914400
30,3.314200
40,2.532600
50,2.368900
60,1.559700
70,1.131400
80,1.189200
90,0.981400
100,0.854400


Step,Training Loss
10,3.213900
20,2.914400
30,3.314200
40,2.532600
50,2.368900
60,1.559700
70,1.131400
80,1.189200
90,0.981400
100,0.854400


TrainOutput(global_step=16312, training_loss=0.7047251285715627, metrics={'train_runtime': 15964.8165, 'train_samples_per_second': 1.022, 'total_flos': 3.931124008413082e+16, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 511313920, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -900087808, 'train_mem_gpu_alloc_delta': 1502172160, 'train_mem_cpu_peaked_delta': 900120576, 'train_mem_gpu_peaked_delta': 5588389888})

In [44]:
trainer.save_model('/content/gdrive/My Drive/relevance_model_gpt')

In [55]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [56]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [57]:
# load model
model = BertForNextSentencePrediction.from_pretrained('/content/gdrive/My Drive/relevance_model_gpt', num_labels=2)
model.eval()

model.to(device)
items = {key: torch.tensor(val).to(device) for key, val in test_encodings.items()}

NameError: ignored

In [ ]:
preds = []
for batch_idxs in tqdm(batch(range(len(items['input_ids'])), 16)):
  batch_input_ids = items['input_ids'][batch_idxs]
  batch_attention_mask = items['attention_mask'][batch_idxs]
  batch_token_type_ids = items['token_type_ids'][batch_idxs]

  outs = model.forward(input_ids=batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids)
  probs = torch.softmax(outs.logits, 1)
  classes = torch.argmax(probs, dim=1)

  preds += list(classes.cpu().numpy())

In [ ]:
sklearn.metrics.f1_score(preds, test_labels, average='weighted')

In [ ]:
sklearn.metrics.confusion_matrix(preds, test_labels)